# Lab 15.4: Reinsurance Networks and Treaty Management

**Duration:** 10 minutes  
**Difficulty:** Advanced  

## Learning Objectives

In this notebook, you will:
- Create reinsurance company partnerships
- Build treaty management structures (Catastrophe, Quota Share, Surplus)
- Implement multi-party reinsurance contracts
- Analyze risk distribution and treaty performance

---

## Step 1: Connect to Neo4j

Establish connection and verify specialty insurance is in place.

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
from datetime import datetime, date
import uuid

# Connect to Neo4j Enterprise instance
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def run_query(query, parameters=None):
    with driver.session(database="insurance") as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

# Verify current database state
current_state = run_query("""
MATCH (n) 
RETURN labels(n)[0] AS node_type, count(n) AS count
ORDER BY count DESC
""")

print("Current Database State:")
for record in current_state:
    print(f"  {record['node_type']}: {record['count']} nodes")

## Step 2: Understanding Reinsurance

### Business Context: Reinsurance Fundamentals

**What is Reinsurance?**
- "Insurance for insurance companies"
- Primary insurer (ceding company) transfers risk to reinsurer
- Allows insurers to write more business than capital alone would permit
- Protects against large losses and catastrophic events
- Stabilizes financial results year over year

**Types of Reinsurance Treaties:**

1. **Catastrophe Excess of Loss**
   - Covers losses exceeding a specified retention (attachment point)
   - Primary use: Protection against natural catastrophes
   - Example: $50M excess of $5M (reinsurer pays losses from $5M to $55M)
   - Reinstatements: Right to restore coverage after loss (at additional premium)

2. **Quota Share**
   - Reinsurer takes fixed percentage of all policies in covered class
   - Pro-rata sharing of premiums and losses
   - Ceding commission paid to primary insurer for acquisition costs
   - Profit commission if loss ratio favorable
   - Example: 25% quota share on commercial lines

3. **Surplus Share**
   - Primary insurer retains amount up to line limit
   - Reinsurer takes surplus above retention, up to treaty capacity
   - Variable cession percentage by policy
   - Common for specialty and professional liability
   - Example: Retain $1M, cede surplus up to $10M treaty capacity

**Financial Strength Ratings:**
- A.M. Best ratings measure reinsurer financial strength
- A++ and A+ = Superior
- A and A- = Excellent
- Ratings critical for regulatory approval and credit risk

In [ ]:
# Create reinsurance companies
reinsurance_companies_query = """
// Create Reinsurance Companies
CREATE (munich_re:ReinsuranceCompany:PartnerOrganization {
  id: randomUUID(),
  company_id: "REIN-001",
  company_name: "Munich Re America",
  reinsurer_type: "Traditional",
  am_best_rating: "A++",
  financial_strength: "Superior",
  geographic_scope: "Global",
  
  // Contact information
  headquarters: "New York, NY",
  regional_office: "Dallas, TX",
  contact_person: "David Richardson",
  contact_title: "Regional Director",
  phone: "214-555-0300",
  email: "drichardson@munichre.com",
  
  // Business metrics
  surplus: 15000000000.00,
  market_share: 0.12,
  specialties: ["Property", "Casualty", "Life", "Specialty"],
  
  created_at: datetime(),
  created_by: "reinsurance_management",
  version: 1
})

CREATE (swiss_re:ReinsuranceCompany:PartnerOrganization {
  id: randomUUID(),
  company_id: "REIN-002", 
  company_name: "Swiss Re Corporate Solutions",
  reinsurer_type: "Traditional",
  am_best_rating: "A+",
  financial_strength: "Superior",
  geographic_scope: "Global",
  
  // Contact information
  headquarters: "Zurich, Switzerland",
  regional_office: "Austin, TX",
  contact_person: "Maria Rodriguez",
  contact_title: "Account Manager",
  phone: "512-555-0400",
  email: "mrodriguez@swissre.com",
  
  // Business metrics
  surplus: 22000000000.00,
  market_share: 0.15,
  specialties: ["Cyber", "Technology", "Professional Lines"],
  
  created_at: datetime(),
  created_by: "reinsurance_management",
  version: 1
})

CREATE (berkshire:ReinsuranceCompany:PartnerOrganization {
  id: randomUUID(),
  company_id: "REIN-003",
  company_name: "Berkshire Hathaway Reinsurance",
  reinsurer_type: "Traditional",
  am_best_rating: "A++",
  financial_strength: "Superior",
  geographic_scope: "Global",
  
  // Contact information
  headquarters: "Omaha, NE",
  regional_office: "Houston, TX",
  contact_person: "Robert Williams",
  contact_title: "Vice President",
  phone: "713-555-0500",
  email: "rwilliams@brk.com",
  
  // Business metrics
  surplus: 85000000000.00,
  market_share: 0.25,
  specialties: ["Catastrophe", "Large Risks", "Alternative Risk"],
  
  created_at: datetime(),
  created_by: "reinsurance_management",
  version: 1
})

RETURN munich_re.company_name AS munich,
       swiss_re.company_name AS swiss,
       berkshire.company_name AS berkshire
"""

company_results = run_query(reinsurance_companies_query)
print("Reinsurance Companies Created:")
for record in company_results:
    print(f"  {record['munich']}")
    print(f"  {record['swiss']}")
    print(f"  {record['berkshire']}")

## Step 3: Create Reinsurance Treaties

### Business Context: Treaty Structure

**Catastrophe Treaty Example:**
- **Coverage**: $50M excess of $5M retention
- **Rate**: 4.5% of subject premium
- **Reinstatements**: 2 at 100% (can restore coverage twice at full premium)
- **Perils**: Wind, Hail, Tornado, Hurricane
- **Territory**: Texas, Oklahoma, Louisiana

**Quota Share Treaty Example:**
- **Cession**: 25% of all commercial lines premiums and losses
- **Commission**: 32% ceding commission on ceded premium
- **Profit Commission**: 15% of underwriting profit
- **Loss Corridor**: 75%-95% (profit commission only if loss ratio in range)

**Surplus Share Treaty Example:**
- **Retention**: $1M per risk
- **Capacity**: $10M total treaty capacity
- **Lines**: Up to 10 lines (10x $1M retention)
- **Coverage**: Professional liability and technology E&O

In [ ]:
# Create reinsurance contracts (treaties)
treaties_query = """
// Create Reinsurance Contracts (Treaties)
CREATE (cat_treaty:ReinsuranceContract {
  id: randomUUID(),
  contract_number: "TREATY-CAT-2024-001",
  treaty_type: "Catastrophe Excess of Loss",
  coverage_line: "Property",
  
  // Treaty terms
  effective_date: date("2024-01-01"),
  expiration_date: date("2024-12-31"),
  coverage_limit: 50000000.00,
  attachment_point: 5000000.00,
  
  // Financial terms
  rate: 0.045,
  minimum_premium: 450000.00,
  maximum_premium: 2250000.00,
  reinstatement_provisions: "2 at 100%",
  
  // Coverage details
  covered_perils: ["Wind", "Hail", "Tornado", "Hurricane"],
  geographic_scope: "Texas, Oklahoma, Louisiana",
  exclusions: ["Flood", "Earthquake", "Nuclear"],
  
  // Performance
  premium_paid: 675000.00,
  claims_paid: 0.00,
  loss_ratio: 0.00,
  profit_commission: 0.15,
  
  created_at: datetime(),
  created_by: "reinsurance_management",
  version: 1
})

CREATE (quota_share:ReinsuranceContract {
  id: randomUUID(),
  contract_number: "TREATY-QS-2024-002",
  treaty_type: "Quota Share",
  coverage_line: "Commercial Lines",
  
  // Treaty terms
  effective_date: date("2024-01-01"),
  expiration_date: date("2024-12-31"),
  cession_percentage: 0.25,
  commission_rate: 0.32,
  
  // Financial terms
  minimum_ceding_commission: 0.28,
  maximum_ceding_commission: 0.35,
  profit_commission: 0.15,
  loss_corridor: "75% - 95%",
  
  // Coverage details
  covered_classes: ["General Liability", "Commercial Property", "Workers Compensation"],
  retention_amount: 100000.00,
  exclusions: ["Asbestos", "Environmental", "Nuclear"],
  
  // Performance year-to-date
  ceded_premium: 1875000.00,
  ceded_losses: 425000.00,
  commission_earned: 600000.00,
  loss_ratio: 0.227,
  
  created_at: datetime(),
  created_by: "reinsurance_management",
  version: 1
})

CREATE (specialty_treaty:ReinsuranceContract {
  id: randomUUID(),
  contract_number: "TREATY-SPEC-2024-003",
  treaty_type: "Surplus Share",
  coverage_line: "Professional Liability",
  
  // Treaty terms
  effective_date: date("2024-01-01"),
  expiration_date: date("2024-12-31"),
  retention_limit: 1000000.00,
  treaty_capacity: 10000000.00,
  
  // Financial terms
  commission_rate: 0.30,
  profit_commission: 0.20,
  loss_ratio_threshold: 0.65,
  sliding_scale_commission: true,
  
  // Coverage details
  covered_classes: ["Technology E&O", "Professional Liability", "Cyber Liability"],
  covered_territories: ["USA", "Canada", "Europe"],
  exclusions: ["Bodily Injury", "Property Damage", "Criminal Acts"],
  
  // Performance
  ceded_premium: 1250000.00,
  ceded_losses: 185000.00,
  commission_earned: 375000.00,
  loss_ratio: 0.148,
  
  created_at: datetime(),
  created_by: "reinsurance_management",
  version: 1
})

RETURN cat_treaty.contract_number AS catastrophe,
       quota_share.contract_number AS quota,
       specialty_treaty.contract_number AS specialty
"""

treaty_results = run_query(treaties_query)
print("\nReinsurance Treaties Created:")
for record in treaty_results:
    print(f"  Catastrophe Treaty: {record['catastrophe']}")
    print(f"  Quota Share Treaty: {record['quota']}")
    print(f"  Specialty Treaty: {record['specialty']}")

## Step 4: Create Reinsurance Partnerships

### Business Context: Multi-Party Reinsurance

**Participation Structures:**
- **Lead Reinsurer**: Primary contact, often largest participation
- **Following Reinsurers**: Additional capacity providers
- **Participation Percentage**: Each reinsurer's share of treaty
- **Line Size**: Dollar amount of capacity provided

**Syndication Benefits:**
- Spreads risk across multiple reinsurers
- Increases total capacity available
- Diversifies credit risk
- Provides competitive pricing

**Example: Catastrophe Treaty Syndicate**
- Total Capacity: $50M
- Munich Re: 40% ($20M)
- Berkshire: 35% ($17.5M)
- Swiss Re: 25% ($12.5M)

In [ ]:
# Create reinsurance relationships
reinsurance_relationships_query = """
// Match treaties and reinsurers
MATCH (cat_treaty:ReinsuranceContract {contract_number: "TREATY-CAT-2024-001"})
MATCH (quota_share:ReinsuranceContract {contract_number: "TREATY-QS-2024-002"})
MATCH (specialty_treaty:ReinsuranceContract {contract_number: "TREATY-SPEC-2024-003"})
MATCH (munich_re:ReinsuranceCompany {company_id: "REIN-001"})
MATCH (swiss_re:ReinsuranceCompany {company_id: "REIN-002"})
MATCH (berkshire:ReinsuranceCompany {company_id: "REIN-003"})

// Create Catastrophe Treaty Relationships
CREATE (cat_treaty)-[:REINSURED_BY {
  participation_percentage: 0.40,
  line_size: 20000000.00,
  contract_role: "Lead Reinsurer"
}]->(munich_re)

CREATE (cat_treaty)-[:REINSURED_BY {
  participation_percentage: 0.35,
  line_size: 17500000.00,
  contract_role: "Following Reinsurer"
}]->(berkshire)

CREATE (cat_treaty)-[:REINSURED_BY {
  participation_percentage: 0.25,
  line_size: 12500000.00,
  contract_role: "Following Reinsurer"
}]->(swiss_re)

// Create Quota Share Relationship
CREATE (quota_share)-[:REINSURED_BY {
  participation_percentage: 1.00,
  line_size: 25000000.00,
  contract_role: "Sole Reinsurer"
}]->(swiss_re)

// Create Specialty Treaty Relationships
CREATE (specialty_treaty)-[:REINSURED_BY {
  participation_percentage: 0.60,
  line_size: 6000000.00,
  contract_role: "Lead Reinsurer"
}]->(swiss_re)

CREATE (specialty_treaty)-[:REINSURED_BY {
  participation_percentage: 0.40,
  line_size: 4000000.00,
  contract_role: "Following Reinsurer"
}]->(munich_re)

RETURN count(*) AS relationships_created
"""

relationship_results = run_query(reinsurance_relationships_query)
print("\nReinsurance Partnerships Established:")
for record in relationship_results:
    print(f"  Total Partnerships: {record['relationships_created']}")

## Step 5: Reinsurance Analysis Queries

Analyze reinsurance networks and treaty performance.

In [ ]:
# Query 1: Complete reinsurance program overview
program_overview_query = """
MATCH (treaty:ReinsuranceContract)-[r:REINSURED_BY]->(reinsurer:ReinsuranceCompany)
WITH treaty,
     count(reinsurer) AS reinsurer_count,
     collect({
       name: reinsurer.company_name,
       participation: r.participation_percentage,
       line_size: r.line_size,
       role: r.contract_role
     }) AS participants
RETURN treaty.contract_number AS contract,
       treaty.treaty_type AS type,
       treaty.coverage_line AS line,
       COALESCE(treaty.coverage_limit, treaty.treaty_capacity) AS capacity,
       reinsurer_count,
       participants
ORDER BY capacity DESC
"""

program_results = run_query(program_overview_query)
print("\nReinsurance Program Overview:")
for record in program_results:
    print(f"\n  Treaty: {record['contract']}")
    print(f"  Type: {record['type']}")
    print(f"  Line: {record['line']}")
    print(f"  Capacity: ${record['capacity']:,.2f}")
    print(f"  Reinsurers: {record['reinsurer_count']}")
    print("  Participants:")
    for participant in record['participants']:
        print(f"    - {participant['name']}: {participant['participation']*100:.0f}% ({participant['role']})")

In [ ]:
# Query 2: Treaty performance metrics
treaty_performance_query = """
MATCH (treaty:ReinsuranceContract)
RETURN treaty.contract_number AS contract,
       treaty.treaty_type AS type,
       COALESCE(treaty.ceded_premium, treaty.premium_paid) AS premium,
       COALESCE(treaty.ceded_losses, treaty.claims_paid) AS losses,
       treaty.loss_ratio AS loss_ratio,
       COALESCE(treaty.commission_earned, 0) AS commission,
       round((COALESCE(treaty.ceded_premium, treaty.premium_paid) - 
              COALESCE(treaty.ceded_losses, treaty.claims_paid) - 
              COALESCE(treaty.commission_earned, 0)), 2) AS net_result
ORDER BY premium DESC
"""

performance_df = pd.DataFrame(run_query(treaty_performance_query))
print("\nTreaty Performance Metrics:")
print(performance_df.to_string(index=False))

In [ ]:
# Query 3: Reinsurer participation analysis
reinsurer_participation_query = """
MATCH (reinsurer:ReinsuranceCompany)<-[r:REINSURED_BY]-(treaty:ReinsuranceContract)
WITH reinsurer,
     count(treaty) AS treaty_count,
     sum(r.line_size) AS total_capacity,
     collect({
       treaty: treaty.contract_number,
       type: treaty.treaty_type,
       role: r.contract_role,
       capacity: r.line_size
     }) AS treaties
RETURN reinsurer.company_name AS reinsurer,
       reinsurer.am_best_rating AS rating,
       treaty_count,
       total_capacity,
       treaties
ORDER BY total_capacity DESC
"""

participation_results = run_query(reinsurer_participation_query)
print("\nReinsurer Participation Analysis:")
for record in participation_results:
    print(f"\n  Reinsurer: {record['reinsurer']}")
    print(f"  A.M. Best Rating: {record['rating']}")
    print(f"  Treaties: {record['treaty_count']}")
    print(f"  Total Capacity: ${record['total_capacity']:,.2f}")
    print("  Treaty Details:")
    for treaty in record['treaties']:
        print(f"    - {treaty['type']}: ${treaty['capacity']:,.2f} ({treaty['role']})")

In [ ]:
# Query 4: Risk distribution analysis
risk_distribution_query = """
MATCH (treaty:ReinsuranceContract)-[r:REINSURED_BY]->(reinsurer:ReinsuranceCompany)
WITH treaty.coverage_line AS coverage_line,
     sum(r.line_size) AS total_capacity,
     count(DISTINCT reinsurer) AS reinsurer_count,
     collect(DISTINCT reinsurer.company_name) AS reinsurers
RETURN coverage_line,
       total_capacity,
       reinsurer_count,
       reinsurers
ORDER BY total_capacity DESC
"""

distribution_df = pd.DataFrame(run_query(risk_distribution_query))
print("\nRisk Distribution by Coverage Line:")
print(distribution_df.to_string(index=False))

In [ ]:
# Query 5: Treaty type analysis
treaty_type_query = """
MATCH (treaty:ReinsuranceContract)
WITH treaty.treaty_type AS treaty_type,
     count(*) AS treaty_count,
     sum(COALESCE(treaty.ceded_premium, treaty.premium_paid)) AS total_premium,
     sum(COALESCE(treaty.ceded_losses, treaty.claims_paid)) AS total_losses,
     avg(treaty.loss_ratio) AS avg_loss_ratio
RETURN treaty_type,
       treaty_count,
       round(total_premium, 2) AS total_premium,
       round(total_losses, 2) AS total_losses,
       round(avg_loss_ratio, 4) AS avg_loss_ratio
ORDER BY total_premium DESC
"""

type_df = pd.DataFrame(run_query(treaty_type_query))
print("\nTreaty Type Analysis:")
print(type_df.to_string(index=False))

## Step 6: Summary

### What You've Accomplished:

- Created **reinsurance company partnerships** with major global reinsurers
- Built **catastrophe excess of loss treaties** for natural disaster protection
- Implemented **quota share treaties** for proportional risk sharing
- Created **surplus share treaties** for specialty and professional liability
- Established multi-party syndication structures with participation percentages
- Analyzed treaty performance, loss ratios, and commission structures

### Key Reinsurance Concepts:

1. **Risk Transfer Mechanisms**: Understanding how insurers transfer catastrophic risk to reinsurers
2. **Treaty Structures**: Different treaty types serve different risk management objectives
3. **Syndication**: Multiple reinsurers share capacity to provide large coverage amounts
4. **Financial Strength**: A.M. Best ratings critical for reinsurer selection and regulatory approval
5. **Commission Structures**: Ceding commissions compensate for acquisition costs, profit commissions align interests
6. **Loss Ratios**: Key performance metric for treaty profitability (losses / premium)

### Reinsurance Program Benefits:

1. **Capital Relief**: Allows writing more business with same capital base
2. **Catastrophe Protection**: Protects against large natural disaster losses
3. **Earnings Stability**: Smooths financial results by ceding volatile risks
4. **Expertise Access**: Reinsurers provide underwriting and claims expertise
5. **Regulatory Compliance**: Meets solvency requirements through risk transfer

---

**Next:** Continue to notebook 05 for Global Operations and Multi-Currency Support.